In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# 드라이버 실행 및 첫 페이지 로딩
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
url = 'https://terms.naver.com/list.naver?cid=59087&categoryId=59087&page=1'
driver.get(url)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "desc")))

# 전체 페이지 크롤링 함수
def all_pages():
    page_number = 1
    while True:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "desc")))
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        results = []
        for li in soup.select('ul.content_list > li'):
            term_tag = li.select_one('strong.title > a')
            desc_tag = li.select_one('p.desc')
            if term_tag and desc_tag:
                term = term_tag.get_text(strip=True)
                desc = desc_tag.get_text(strip=True)
                results.append({'용어': term, '설명': desc})

        yield results

        # 다음 페이지로 이동
        next_page_num = page_number + 1
        next_link = soup.select_one(f'div#paginate a[href*="page={next_page_num}"]')
        if next_link and 'href' in next_link.attrs:
            next_url = "https://terms.naver.com" + next_link['href']
            driver.get(next_url)
            page_number = next_page_num
        else:
            break

# 크롤링 실행 및 CSV 저장
all_data = []
for page_data in all_pages():
    all_data.extend(page_data)

df = pd.DataFrame(all_data)
df.to_csv('무기의세계.csv', index=False, encoding='utf-8-sig')

print(f"총 {len(df)}개의 용어가 '무기의세계.csv' 파일에 저장되었습니다.")

총 388개의 용어가 '무기의세계.csv' 파일에 저장되었습니다.


In [3]:
wp = pd.read_csv('무기의세계.csv')
wp.head()

,용어,설명
0,F-2 [Mitsubishi F-2],F-2는 일본 항공자위대가 운용중인 전투기로 F-16 전투기를 바탕으로 미국과 일본...
1,알레이버크급 구축함 [Arleigh Burke class destroyer],알레이버크급 구축함은 미 해군의 주력 수상전투함으로 이지스함의 대명사로 알려져 있다...
2,Ju 52 수송기,Ju 52는 제2차 대전 관련 에피소드에서 약방의 감초처럼 등장하는 숨은 주인공이었...
3,X-37B [X-37B Orbital Test Vehicle],X-37B는 미 공군이 운용중인 무인우주선으로 궤도시험기라는 정식명칭을 가지고 있다...
4,미니트맨 [LGM-30 Minuteman],미니트맨은 미 공군이 운용중인 대륙간탄도미사일로 미국의 핵 억제력에 중추적인 위치를...


In [4]:
wp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   용어      388 non-null    object
 1   설명      388 non-null    object
dtypes: object(2)
memory usage: 6.2+ KB


In [6]:
import re
import pandas as pd

# 1. 전처리 함수 정의
def clean_term(term: str) -> str:
    term = re.sub(r"\(.*?\)", "", term)  # 소괄호 제거
    term = re.sub(r"\[.*?\]", "", term)  # 대괄호 제거
    return term.strip()

# 2. CSV 불러오기
wp = pd.read_csv("무기의세계.csv")

# 3. '용어' 컬럼 덮어쓰기
wp["용어"] = wp["용어"].apply(clean_term)

# 4. 덮어쓴 상태로 저장 (원래 파일에 덮어쓰기)
wp.to_csv("무기의세계.csv", index=False)